In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import wandb

c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare Data

In [2]:
train_en_path = "./data_sources/train/train_en.csv"
test_en_path = "./data_sources/test/test_en.csv"

train_it_path = "./data_sources/train/train_it.csv"
test_it_path = "./data_sources/test/test_it.csv"

train_es_path = "./data_sources/train/train_es.csv"
test_es_path = "./data_sources/test/test_es.csv"

train_df = pd.read_csv(train_it_path)
test_df = pd.read_csv(test_it_path)

# convert to lists
X_train_texts = train_df['text'].tolist()
y_train = train_df['label'].tolist()


# test set
X_test_texts = test_df['text'].tolist()
y_test = test_df['label'].tolist()

### Encode Texts

In [3]:
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
embedder = SentenceTransformer(model_name)

# Compute embeddings
X_train = embedder.encode(X_train_texts, convert_to_numpy=True)
X_test  = embedder.encode(X_test_texts, convert_to_numpy=True)

In [4]:
wandb.init(project="reappropriation-logreg", name="logreg_pipeline")

wandb: Currently logged in as: sravisconti (sravisconti-projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### Hyperparameter Tuning

In [5]:
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(
    LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    param_grid,
    scoring="f1_macro",
    cv=5
)
grid.fit(X_train, y_train)

best_clf = grid.best_estimator_

# Log best hyperparameters
wandb.config.update(grid.best_params_)

### Train final model and evaluate

In [6]:
# train final model
best_clf.fit(X_train, y_train)

# evaluate on test set
y_pred = best_clf.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred)

class_names = ["offensive", "reappropriative"]

cm_df = pd.DataFrame(
    confusion_matrix,
    index=[f"True: {c}" for c in class_names],
    columns=[f"Pred: {c}" for c in class_names]
)

# Log metrics
wandb.log({
    "f1_macro": f1,
    "accuracy": accuracy,
    "classification_report": report,
    "confusion_matrix": confusion_matrix
})

print(f"Test F1 Macro: {f1}")
print(f"Test Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm_df)

Test F1 Macro: 0.8721407624633432
Test Accuracy: 0.9128440366972477
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       176
           1       0.72      0.90      0.80        42

    accuracy                           0.91       218
   macro avg       0.85      0.91      0.87       218
weighted avg       0.93      0.91      0.92       218

Confusion Matrix:
                       Pred: offensive  Pred: reappropriative
True: offensive                    161                     15
True: reappropriative                4                     38
